In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
accident = pd.read_excel('data/accident_total_2016-2020.xlsx', sheet_name = '편집_1')

In [3]:
conzone = pd.read_csv('data/info_total.csv', encoding = 'cp949')

In [4]:
start_end_point = conzone.iloc[:, [5, 6, 7, 8, 9, 10]].drop_duplicates()
start_end_point.head(2)

,도로명,도로단축명,도로표출명,기점종점방향구분코드,시점명,종점명
0,경부선,경부선,경부,E,부산,서울
112,경부선,경부선,경부,S,부산,서울


In [5]:
total_df = pd.merge(accident, start_end_point, how = 'left', left_on = ['노선명', '방향'], right_on = ['도로단축명', '종점명']).dropna(subset = ['도로명', '도로단축명', '도로표출명', '기점종점방향구분코드'])

In [6]:
conzone.head(2)

,Unnamed: 0,콘존ID,콘존길이,콘존명,노선번호,도로명,도로단축명,도로표출명,기점종점방향구분코드,시점명,...,시작노드ID,종료노드ID,시작노드_도로이정,종료노드_도로이정,차로수,버스전용차로유무,도로등급구분코드,제한속도,소통원활기준속도,정체기준속도
0,0,0010CZE005,200.0,경부고속국도시점-구서IC,10,경부선,경부선,경부,E,부산,...,491,4,0.0,0.20,3.0,0,101,100.0,80,40
1,1,0010CZE010,1820.0,구서IC-영락IC,10,경부선,경부선,경부,E,부산,...,4,446,0.2,2.02,3.0,0,101,100.0,80,40


In [7]:
total_df.head(3)

,사고일자,사고일자.1,월별구분,사고시간,시간단위_3시간,주야구분,노선명,이정,방향,사망,...,절성토구분,방책시설_중분대,원인차차종,원인차_차종구분,도로명,도로단축명,도로표출명,기점종점방향구분코드,시점명,종점명
1,2016-01-01,20160101,01월,18:40:00,18 ~21,야간,광주대구선,141.7,대구,0,...,평지,가드레일,승용,중형,광주대구선,광주대구선,광주대구,E,광주,대구
2,2016-01-01,20160101,01월,18:40:00,18 ~21,야간,광주대구선,141.7,대구,0,...,평지,가드레일,승용,중형,광주대구선,광주대구선,광주대구,S,광주,대구
3,2016-01-02,20160102,01월,08:15:00,6 ~9,주간,남해선,146.5,부산,0,...,평지,고정식방호벽(127cm),화물,소형,남해선(순천-부산),남해선,남해,E,순천,부산


In [13]:
total_df['도로명']

'광주대구선'

In [18]:
save_total_df_dir = 'data/accident_merged.csv'

In [16]:
# 도로단축명, 기종점방향구분코드가 conzone과 total_df가 같을 경우,
# total_df의 '이정'이 conzone의 시작노드_도로이정과 종료노드_도로이정 사이에 있으면 해당 콘존을 입력
# 기종점방향구분코드 : E이면 시작노드 < 종료노드, S이면 시작노드 > 종료노드임.

total_df['conzoneID'] = None #새로운 비어있는 컬럼 : 콘존ID
total_df['conzoneName'] = None # 콘존명

for i in tqdm(range(len(total_df))): # total_df의 i번째 행에 대하여
    
    # 먼저, total_df[i]와 같은 도로단축명, 기종점 방향구분코드를 가진 conzone만 걸러낸다.
    i_roadname = total_df['도로명'].iloc[i]
    i_direction = total_df['기점종점방향구분코드'].iloc[i]

    czdf = conzone[(conzone['도로명'] == i_roadname) & (conzone['기점종점방향구분코드'] == i_direction)]
    
    # i_direction이 E냐 S냐에 따라서 다른 if 절을 적용
    # conzone의 시작노드_도로이정, 종료노드_도로이정 사이에 '도로이정'이 있을 시 해당 콘존을 투입
    location = total_df['이정'].iloc[i]
    
    for j in range(len(czdf)):
        start_location = czdf['시작노드_도로이정'].iloc[j]
        end_location = czdf['종료노드_도로이정'].iloc[j]
        
        if i_direction == 'E':
            if location >= start_location and location < end_location:
                conzoneID = czdf['콘존ID'].iloc[j]
                conzoneName = czdf['콘존명'].iloc[j]
                
            else:
                pass
                
        elif i_direction == 'S':
            if location <= start_location and location > end_location:
                conzoneID = czdf['콘존ID'].iloc[j]
                conzoneName = czdf['콘존명'].iloc[j]
                
            else:
                pass
            
        else:
            pass
    
    total_df['conzoneID'].iloc[i] = conzoneID
    total_df['conzoneName'].iloc[i] = conzoneName

total_df.to_csv(save_total_df_dir, encoding = 'cp949')
print('PROCESSING FINISHED')

  0%|                                                                                         | 0/3796 [00:00<?, ?it/s]C:\dev\Anaconda_3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|█████████████████████████████████████████████████████████████████████████████| 3796/3796 [00:09<00:00, 412.32it/s]

PROCESSING FINISHED
